# Regular Languages and Automata


**Objectives:**
1. Familiarize with graph representation of finite automata. 

2. Implement basic operations on finite automaton. 

3. Practice elementary proof techniques.


**Reading:** Chap. 1. 


**Questions:**
1. What is a state in finite automaton? What is a `dead` state? In our work, we generally refer to a dead state as a `sink` state.

2. What is an alphabet? What is a string? What is a language?

3. What is an incomplete automaton? How to complete an incomplete automaton?

4. What is a regular language? What other types of languages exist? 

In [1]:
import sys
sys.path.extend(['/home/ggsolver', '/home/cirl-assignments'])

import ggsolver

In [2]:
import logging
logging.basicConfig(format="%(levelname)s:%(filename)s:%(lineno)d:%(message)s",
                   level=logging.INFO)

## DFA in `ggsolver`

`ggsolver` is a python library containing set-based solvers for synthesis of winning strategies in two-player games on graphs. In these games, we use different types of automaton to capture the progress a player has made towards completing its task. 

`ggsolver` has two packages:
* `ggsolver`: defines different types of games and their solvers.
* `parsers`: defines different formal languages, their grammars and automata theoretic representations. 

In this notebook, we will focus on `parsers.automata` module that defines `Dfa` class. 


### Constructing a DFA.

A `Dfa` object can be created in two ways. 

* `EXPLICIT`: `Dfa` object is created using a `networkx.MultiDiGraph` object. The graph must have edges labeled with `symbol` attribute that represents the set of atomic propositions that must be true to enable the edge. 

* `SYMBOLIC`: `Dfa` object is created by defining the components of $\langle Q, \Sigma, \delta, q_0, F \rangle$ in addition to the predecessor and successor functions. 


**Example 1 (EXPLICIT mode).** Consider a DFA with $\Sigma = \{0, 1\}$ that accepts any word that has `00`.

In [3]:
# Importing `parsers` provides access to Dfa class.
import parsers
import networkx as nx

# Define `Dfa` graph
graph = nx.MultiDiGraph()
graph.add_nodes_from(["No 0", "One 0", "Two 0"])
graph.add_edges_from([
    ("No 0", "One 0", {"symbol": "0"}),
    ("No 0", "No 0", {"symbol": "1"}),
    ("One 0", "Two 0", {"symbol": "0"}),
    ("One 0", "No 0", {"symbol": "1"}),
    ("Two 0", "Two 0", {"symbol": "0"}),
    ("Two 0", "Two 0", {"symbol": "1"}),
])

# Instantiate a Dfa object
dfa_exp = parsers.Dfa(name="00-language-explicit")
dfa_exp.construct_explicit(graph=graph, init_st="No 0", final={"Two 0"})

print(str(dfa_exp))


    This Dfa has 3 states.
    Alphabet: {'1', '0'}
    Starting state: No 0
    Accepting states: {'Two 0'}
    States: ['No 0', 'One 0', 'Two 0']
    Transition function:
		No 0 -- {} --> None
		No 0 -- {'1'} --> No 0
		No 0 -- {'0'} --> One 0
		No 0 -- {'1', '0'} --> None
		One 0 -- {} --> None
		One 0 -- {'1'} --> No 0
		One 0 -- {'0'} --> Two 0
		One 0 -- {'1', '0'} --> None
		Two 0 -- {} --> None
		Two 0 -- {'1'} --> Two 0
		Two 0 -- {'0'} --> Two 0
		Two 0 -- {'1', '0'} --> None


**Example 2 (SYMBOLIC mode).** Let us see how the same DFA can be constructed in `SYMBOLIC` mode.

In [4]:
# Define components of Dfa tuple
states = ["No 0", "One 0", "Two 0"]
alphabet = {"0", "1"}
init_st = "No 0"
final = {"Two 0"}

def delta(state, true_atoms):
    if state == "No 0":
        if set(true_atoms) == {"0"}:
            return "One 0"
        else: 
            return "No 0"
    
    elif state == "One 0":
        if set(true_atoms) == {"0"}:
            return "Two 0"
        elif set(true_atoms) == set():
            return "One 0"
        else: 
            return "No 0"
    
    elif state == "Two 0":
        return "Two 0"


def pred(state):
    if state == "No 0":
        return {
            ("No 0", {"1"}),
            ("No 0", set()),
            ("One 0", {"1"})
        }
    
    elif state == "One 0":
        return {
            ("No 0", {"0"}),
            ("One 0", set())
        }
    
    elif state == "Two 0":
        return {
            ("Two 0", {"1"}),
            ("Two 0", {"0"}),
            ("Two 0", set()),
            ("One 0", {"0"})
        }


def succ(state):
    if state == "No 0":
        return {
            ("No 0", {"1"}),
            ("No 0", set()),
            ("One 0", {"0"})
        }
    
    elif state == "One 0":
        return {
            ("No 0", {"1"}),
            ("One 0", set())
            ("Two 0", {"0"}),
        }
    
    elif state == "Two 0":
        return {
            ("Two 0", {"1"}),
            ("Two 0", {"0"}),
            ("Two 0", set()),
        }


# Construct Dfa object
dfa_sym = parsers.Dfa(name="00-language-symbolic")
dfa_sym.construct_symbolic(states=states, alphabet=alphabet, delta=delta, pred=pred, succ=succ, init_st=init_st, final=final)

print(str(dfa_sym))


    This Dfa has 3 states.
    Alphabet: {'1', '0'}
    Starting state: No 0
    Accepting states: {'Two 0'}
    States: ['No 0', 'One 0', 'Two 0']
    Transition function:
		No 0 -- {} --> No 0
		No 0 -- {'1'} --> No 0
		No 0 -- {'0'} --> One 0
		No 0 -- {'1', '0'} --> No 0
		One 0 -- {} --> One 0
		One 0 -- {'1'} --> No 0
		One 0 -- {'0'} --> Two 0
		One 0 -- {'1', '0'} --> No 0
		Two 0 -- {} --> Two 0
		Two 0 -- {'1'} --> Two 0
		Two 0 -- {'0'} --> Two 0
		Two 0 -- {'1', '0'} --> Two 0


<div class="alert alert-block alert-warning">
<b>Remark:</b> Symbolic mode is very useful when defining operations over two `Dfa` objects such as union or intersection products. 
</div>

## Exercise 1 (Completing DFA)

In example 1, several warnings are raised due to *incompleteness of automaton*. Implement the following functions 
* to check whether a given `Dfa` is complete.
* to complete an incomplete `Dfa`. 

**Tips:**

* `Dfa.states()` returns a generator over states in `Dfa`.
* `Dfa.pred(v)` returns a set of (u, symbol)-pairs such that state `v` can be reached from `u` given set of true atoms `symbol`. 
* `Dfa.succ(u)` returns a set of (v, symbol)-pairs such that state `v` can be reached from `u` given set of true atoms `symbol`. 
* `Dfa.delta(v)` returns a state `v` that is reached from `u` given set of true atoms `symbol`. 

In [5]:
def is_complete(dfa):
    """
    Return `True` if complete, otherwise `False`.
    """
    pass


def complete(dfa):
    """
    Completes an incomplete automaton.
    """
    pass


# Check output on example 1.
print(is_complete(dfa_exp))
print(str(complete(dfa_exp)))


None
None


## Exercise 2 (Sink State)

In example 2, although `dfa_sym` is complete, it's behavior may not be as expected. Suppose that the input to DFA is from a hardware circuit, where input can either be "0" or "1". Thus, any `{}` or `{"0", "1"}` inputs are to be labeled as errorneous inputs. We may do this by introducing a `sink` state and redirecting transitions with any unexpected inputs to `sink`. 

Modify `delta, pred and succ` functions for `dfa_sym` to have a sink state and redirect all `{}` or `{"0", "1"}` transitions into sink. 



In [6]:
def delta2(state, true_atoms):
    pass

def pred2(v):
    pass

def succ2(u):
    pass


dfa_00 = parsers.Dfa(name="00-language-symbolic")
dfa_00.construct_symbolic(states, alphabet, delta2, pred2, succ2, init_st, final, skip_validation=True)
print(str(dfa_00))


    This Dfa has 3 states.
    Alphabet: {'1', '0'}
    Starting state: No 0
    Accepting states: {'Two 0'}
    States: ['No 0', 'One 0', 'Two 0']
    Transition function:
		No 0 -- {} --> None
		No 0 -- {'1'} --> None
		No 0 -- {'0'} --> None
		No 0 -- {'1', '0'} --> None
		One 0 -- {} --> None
		One 0 -- {'1'} --> None
		One 0 -- {'0'} --> None
		One 0 -- {'1', '0'} --> None
		Two 0 -- {} --> None
		Two 0 -- {'1'} --> None
		Two 0 -- {'0'} --> None
		Two 0 -- {'1', '0'} --> None


**Note 1:** Once a `Dfa` is constructed, it cannot be reconstructed. We must create a new instance of object to reconstruct it. 

**Note 2:** `skip_validation` input skips validation while constructing the `Dfa`.


## Exercise 3 (Product of DFA)

Define a complete `Dfa`, say `dfa_11`, with $\Sigma = \{"0", "1"\}$ that accepts any word containing `11`. 
Given `dfa_00, dfa_11`, construct a `Dfa` that accepts any word containing `00` or `11`. Determine the type of product operation that must be used and implement it. 

In [12]:
dfa_11 = parsers.Dfa(name="11-language")

In [13]:
def product(dfa1, dfa2):
    dfa = parsers.Dfa(name=f"{repr(dfa1)} x {repr(dfa2)}")
    
    # Implement.
    
    return dfa


prod_dfa = product(dfa_00, dfa_11)
print(str(prod_dfa))

NameError: name 'parser' is not defined

## Exercise 4 (Plotting)

It's easier to gain insight if we can visualize the `Dfa` graph. Use `networkx.draw` tool to plot the product `Dfa`. 


**Tips:**
* If you constructed product DFA in symbolic mode, then graph must be made explicit before plotting. This is done using `Dfa.make_explicit` function. This function will requires complete `Dfa`. Also, `make_explicit` changes the `Dfa` mode to `EXPLICIT`. 

* Given an explicit DFA, `Dfa.graph` property will return the `networkx.MultiDiGraph` object which may be used for plotting. 


In [ ]:
def draw(dfa):
    pass